In [1]:
import pandas as pd
import re
df1=pd.read_csv('/content/1990 - 1999.csv')
df2=pd.read_csv('/content/2000 - Feb 2012.csv')
df3=pd.read_csv('/content/Jan 2015 to Dec 2016.csv')
df4=pd.read_csv('/content/Jan-2017 onwards.csv')
df5=pd.read_csv('/content/Mar 2012 to Dec 2014.csv')
df=pd.concat([df1,df2,df3,df4,df5],ignore_index=False)
df['month']=pd.to_datetime(df['month'])
df['lease_commence_date']=pd.to_datetime(df['lease_commence_date'])
def parse_duration(duration):
  if isinstance(duration,str):
    match=re.match(r"(\d+)\s+years\s+(\d+)\s+months",duration)
    if match:
      years=int(match.group(1))
      months=int(match.group(2))
      days=years*365+months*30
      return int(days)
  return pd.NaT
df['remaining_lease']=df['remaining_lease'].apply(parse_duration)
print(df.dtypes)
df['remaining_lease'] = df['remaining_lease'].fillna((df['lease_commence_date'] + pd.DateOffset(years=99)) - df['month'])
df['remaining_lease']=df['remaining_lease'].astype('str')
def convert_to(days):
  days=days.split()
  return int(days[0])
df['remaining_lease']=df['remaining_lease'].apply(convert_to)
def dayconv(days):
  yrs=days//365
  days=days%365
  months=days//30
  return f'{yrs} years {months} months'
df['remaining_lease']=df['remaining_lease'].apply(dayconv)
df['lease_commence_date']=pd.to_datetime(df['lease_commence_date']).dt.date
df['month']=pd.to_datetime(df['month']).dt.date



month                  datetime64[ns]
town                           object
flat_type                      object
block                          object
street_name                    object
storey_range                   object
floor_area_sqm                float64
flat_model                     object
lease_commence_date    datetime64[ns]
resale_price                  float64
remaining_lease                object
dtype: object


In [2]:
import requests
import json
list_of_mrt = [
    'Jurong East MRT Station',
    'Bukit Batok MRT Station',
    'Bukit Gombak MRT Station',
    'Choa Chu Kang MRT Station',
    'Yew Tee MRT Station',
    'Kranji MRT Station',
    'Marsiling MRT Station',
    'Woodlands MRT Station',
    'Admiralty MRT Station',
    'Sembawang MRT Station',
    'Canberra MRT Station',
    'Yishun MRT Station',
    'Khatib MRT Station',
    'Yio Chu Kang MRT Station',
    'Ang Mo Kio MRT Station',
    'Bishan MRT Station',
    'Braddell MRT Station',
    'Toa Payoh MRT Station',
    'Novena MRT Station',
    'Newton MRT Station',
    'Orchard MRT Station',
    'Somerset MRT Station',
    'Dhoby Ghaut MRT Station',
    'City Hall MRT Station',
    'Raffles Place MRT Station',
    'Marina Bay MRT Station',
    'Marina South Pier MRT Station',
    'Pasir Ris MRT Station',
    'Tampines MRT Station',
    'Simei MRT Station',
    'Tanah Merah MRT Station',
    'Bedok MRT Station',
    'Kembangan MRT Station',
    'Eunos MRT Station',
    'Paya Lebar MRT Station',
    'Aljunied MRT Station',
    'Kallang MRT Station',
    'Lavender MRT Station',
    'Bugis MRT Station',
    'Tanjong Pagar MRT Station',
    'Outram Park MRT Station',
    'Tiong Bahru MRT Station',
    'Redhill MRT Station',
    'Queenstown MRT Station',
    'Commonwealth MRT Station',
    'Buona Vista MRT Station',
    'Dover MRT Station',
    'Clementi MRT Station',
    'Chinese Garden MRT Station',
    'Lakeside MRT Station',
    'Boon Lay MRT Station',
    'Pioneer MRT Station',
    'Joo Koon MRT Station',
    'Gul Circle MRT Station',
    'Tuas Crescent MRT Station',
    'Tuas West Road MRT Station',
    'Tuas Link MRT Station',
    'Expo MRT Station',
    'Changi Airport MRT Station',
    'HarbourFront MRT Station',
    'Chinatown MRT Station',
    'Clarke Quay MRT Station',
    'Little India MRT Station',
    'Farrer Park MRT Station',
    'Boon Keng MRT Station',
    'Potong Pasir MRT Station',
    'Woodleigh MRT Station',
    'Serangoon MRT Station',
    'Kovan MRT Station',
    'Hougang MRT Station',
    'Buangkok MRT Station',
    'Sengkang MRT Station',
    'Punggol MRT Station',
    'Bras Basah MRT Station',
    'Esplanade MRT Station',
    'Promenade MRT Station',
    'Nicoll Highway MRT Station',
    'Stadium MRT Station',
    'Mountbatten MRT Station',
    'Dakota MRT Station',
    'MacPherson MRT Station',
    'Tai Seng MRT Station',
    'Bartley MRT Station',
    'Lorong Chuan MRT Station',
    'Marymount MRT Station',
    'Caldecott MRT Station',
    'Botanic Gardens MRT Station',
    'Farrer Road MRT Station',
    'Holland Village MRT Station',
    'one-north MRT Station',
    'Kent Ridge MRT Station',
    'Haw Par Villa MRT Station',
    'Pasir Panjang MRT Station',
    'Labrador Park MRT Station',
    'Telok Blangah MRT Station',
    'Bayfront MRT Station',
    'Bukit Panjang MRT Station',
    'Cashew MRT Station',
    'Hillview MRT Station',
    'Beauty World MRT Station',
    'King Albert Park MRT Station',
    'Sixth Avenue MRT Station',
    'Tan Kah Kee MRT Station',
    'Stevens MRT Station',
    'Rochor MRT Station',
    'Downtown MRT Station',
    'Telok Ayer MRT Station',
    'Fort Canning MRT Station',
    'Bencoolen MRT Station',
    'Jalan Besar MRT Station',
    'Bendemeer MRT Station',
    'Geylang Bahru MRT Station',
    'Mattar MRT Station',
    'Ubi MRT Station',
    'Kaki Bukit MRT Station',
    'Bedok North MRT Station',
    'Bedok Reservoir MRT Station',
    'Tampines West MRT Station',
    'Tampines East MRT Station',
    'Upper Changi MRT Station',
    'Woodlands North MRT Station',
    'Woodlands South MRT Station'
]
mrt_lat=[]
mrt_lon=[]
mrt_lat = []
mrt_long = []

for i in range(0, len(list_of_mrt)):
    query_address = list_of_mrt[i]
    query_string = "https://www.onemap.gov.sg/api/common/elastic/search?searchVal="+str(query_address)+"&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    resp = requests.get(query_string)
    data_mrt=json.loads(resp.content)
    if data_mrt['found'] != 0:
        mrt_lat.append(data_mrt["results"][0]["LATITUDE"])
        mrt_long.append(data_mrt["results"][0]["LONGITUDE"])
    else:
        mrt_lat.append('NotFound')
        mrt_lat.append('NotFound')
        print ("No Results")

In [3]:
mrt_location = pd.DataFrame({
    'MRT': list_of_mrt,
    'latitude': mrt_lat,
    'longitude': mrt_long
})
df['address'] = df['block'] + " " + df['street_name']



In [4]:
len(df['address'].unique())

9849

In [5]:
address_list = df['address'].unique()
latitude = []
longitude = []
blk_no = []
road_name = []
postal_code = []
address = []
count = 0

for row in range(len(address_list)):
    query_address = address_list[row]
    query_string="https://www.onemap.gov.sg/api/common/elastic/search?searchVal="+str(query_address)+"&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    resp = requests.get(query_string)


    data_geo_location=json.loads(resp.content)
    if data_geo_location['found'] != 0:
        latitude.append(data_geo_location['results'][0]['LATITUDE'])
        longitude.append(data_geo_location['results'][0]['LONGITUDE'])
        blk_no.append(data_geo_location['results'][0]['BLK_NO'])
        road_name.append(data_geo_location['results'][0]['ROAD_NAME'])
        postal_code.append(data_geo_location['results'][0]['POSTAL'])
        address.append(query_address)
    else:
        print ("No Results")

In [6]:
df_coordinates = pd.DataFrame({
    'latitude': latitude,
    'longitude': longitude,
    'blk_no': blk_no,
    'road_name': road_name,
    'postal_code': postal_code,
    'address': address
})
len(df_coordinates)
list_of_lat = df_coordinates['latitude']
list_of_long = df_coordinates['longitude']
mrt_lat = mrt_location['latitude']
mrt_long = mrt_location['longitude']
list_of_coordinates = []
list_of_mrt_coordinates = []

for lat, long in zip(list_of_lat, list_of_long):
    list_of_coordinates.append((lat,long))
for lat, long in zip(mrt_lat, mrt_long):
    list_of_mrt_coordinates.append((lat, long))
from geopy.distance import geodesic

list_of_dist_mrt = []
min_dist_mrt = []

for origin in list_of_coordinates:
    for destination in range(0, len(list_of_mrt_coordinates)):
        list_of_dist_mrt.append(geodesic(origin,list_of_mrt_coordinates[destination]).meters)
    shortest = (min(list_of_dist_mrt))
    min_dist_mrt.append(shortest)
    list_of_dist_mrt.clear()
# Distance from CDB
cbd_dist = []

for origin in list_of_coordinates:
    cbd_dist.append(geodesic(origin,(1.2830, 103.8513)).meters)
# Put MRT and CBD distance together
df_coordinates['cbd_dist'] = cbd_dist
df_coordinates['min_dist_mrt'] = min_dist_mrt
df_coordinates.to_csv('df_coordinates.csv',index=False)
df_coordinates = pd.read_csv('df_coordinates.csv')
df_new = df_coordinates.merge(df, on="address", how='outer')
df_new['resale_price'] = df_new['resale_price'].astype('float')
df_new['floor_area_sqm'] = df_new['floor_area_sqm'].astype('float')
df_new.dropna(inplace=True)
df_new.to_csv('combined.csv', index = False)
df_com=pd.read_csv('combined.csv')
df_com.head()
df_com.isnull().sum()

<ipython-input-6-a8b5bbc8dde0>:47: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_com=pd.read_csv('combined.csv')


,0
latitude,0
longitude,0
blk_no,0
road_name,0
postal_code,0
address,0
cbd_dist,0
min_dist_mrt,0
month,0
town,0


In [7]:
df_com.dtypes

,0
latitude,float64
longitude,float64
blk_no,object
road_name,object
postal_code,object
address,object
cbd_dist,float64
min_dist_mrt,float64
month,object
town,object
